### Sentiment Analysis on Amazon Reviews

In [83]:
#loading the required libraries

In [84]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras import models, layers, optimizers
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
import bz2
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import re

%matplotlib inline



In [85]:
#Creating a function to load the text and labels from train and test set

In [88]:
def get_labels_and_texts(file):
    labels = []
    texts = []
    for line in bz2.BZ2File(file):
        x = line.decode("utf-8")
        labels.append(int(x[9]) - 1)
        texts.append(x[10:].strip())
    return np.array(labels), texts
train_labels, train_texts = get_labels_and_texts('train.ft.txt.bz2')
test_labels, test_texts = get_labels_and_texts('test.ft.txt.bz2')

In [89]:
train_labels[3]

1

In [90]:
train_texts[3]

"Excellent Soundtrack: I truly like this soundtrack and I enjoy video game music. I have played this game and most of the music on here I enjoy and it's truly relaxing and peaceful.On disk one. my favorites are Scars Of Time, Between Life and Death, Forest Of Illusion, Fortress of Ancient Dragons, Lost Fragment, and Drowned Valley.Disk Two: The Draggons, Galdorb - Home, Chronomantique, Prisoners of Fate, Gale, and my girlfriend likes ZelbessDisk Three: The best of the three. Garden Of God, Chronopolis, Fates, Jellyfish sea, Burning Orphange, Dragon's Prayer, Tower Of Stars, Dragon God, and Radical Dreamers - Unstealable Jewel.Overall, this is a excellent soundtrack and should be brought by those that like video game music.Xander Cross"

In [63]:
train_labels=train_labels[0:500]

In [64]:
train_texts=train_texts[0:500]

In [65]:
#text pre-processing

In [75]:
import re
NON_ALPHANUM = re.compile(r'[\W]')
NON_ASCII = re.compile(r'[^a-z0-1\s]')
def normalize_texts(texts):
    normalized_texts = []
    for text in texts:
        lower = text.lower()
        no_punctuation = NON_ALPHANUM.sub(r' ', lower)
        no_non_ascii = NON_ASCII.sub(r'', no_punctuation)
        normalized_texts.append(no_non_ascii)
    return normalized_texts
        
train_texts = normalize_texts(train_texts)
test_texts = normalize_texts(test_texts)

In [76]:
train_texts[3]

'excellent soundtrack  i truly like this soundtrack and i enjoy video game music  i have played this game and most of the music on here i enjoy and it s truly relaxing and peaceful on disk one  my favorites are scars of time  between life and death  forest of illusion  fortress of ancient dragons  lost fragment  and drowned valley disk two  the draggons  galdorb   home  chronomantique  prisoners of fate  gale  and my girlfriend likes zelbessdisk three  the best of the three  garden of god  chronopolis  fates  jellyfish sea  burning orphange  dragon s prayer  tower of stars  dragon god  and radical dreamers   unstealable jewel overall  this is a excellent soundtrack and should be brought by those that like video game music xander cross'

In [77]:
#countvectorizer

In [78]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary=True)
cv.fit(train_texts)
X = cv.transform(train_texts)
X_test = cv.transform(test_texts)

In [79]:
X_test

<400000x5778 sparse matrix of type '<class 'numpy.int64'>'
	with 18647020 stored elements in Compressed Sparse Row format>

In [71]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


X_train, X_val, y_train, y_val = train_test_split(
    X, train_labels, train_size = 0.75
)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_val, lr.predict(X_val))))

Accuracy for C=0.01: 0.768
Accuracy for C=0.05: 0.8
Accuracy for C=0.25: 0.8
Accuracy for C=0.5: 0.8
Accuracy for C=1: 0.76


In [80]:
lr.predict(X_test[34])

array([0])

In [81]:
test_labels[34]

1

In [82]:
test_texts[34]

'they d watch it nonstop if i d let them  apart from it arriving broken the first time  my kids are loving this dvd  which i alternate with the other i purchased to keep my own sanity  this dvd has created a thomas fan out of my  1  yr  old daughter as well '